<a href="https://colab.research.google.com/github/MarcelloCeresini/ChessBreaker/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
username = 'MarcelloCeresini'
repository = 'ChessBreaker'

In [ ]:
# COLAB ONLY CELLS
try:
    import google.colab
    IN_COLAB = True
    !nvidia-smi             # Check which GPU has been chosen for us
    !rm -rf logs
    #from google.colab import drive
    #drive.mount('/content/drive')
    #%cd /content/drive/MyDrive/GitHub/
    !git clone https://github.com/{username}/{repository}.git
    %cd {repository}
    %ls
except:
    IN_COLAB = False

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Cloning into 'ChessBreaker'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 28 (delta 12), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (28/28), done.
/content/ChessBreaker/ChessBreaker
main.ipynb  README.md  utils.py


In [ ]:
import tensorflow as tf
import chess

from utils import plane_dict
from config import Config

conf = Config()
board = chess.Board()

# N_PLANES = len(plane_dict)
# BOARD_SHAPE = (8, 8)
# BOARD_SIZE = BOARD_SHAPE[0] * BOARD_SHAPE[1]
# MAX_MOVE_COUNT = 255
# N_PIECE_TYPES = 6
# PAST_TIMESTEPS = 8


legal_moves = board.legal_moves


SyntaxError: ignored

In [ ]:
def uniform_tensor(x):
    return tf.fill(conf.BOARD_SHAPE, x)

def normal_input_planes(board):
    normal_planes = tf.zeros((*conf.BOARD_SHAPE, (conf.N_PIECE_TYPES+1)*conf.PAST_TIMESTEPS*2), dtype=tf.dtypes.int8) # adding 1 plane for repetition counter for each timestep for each player
    ###### to add planes here
    tf.cast(
        
    )
def special_input_planes(board):
    return tf.vectorized_map( 
            uniform_tensor, 
            [
                bool(board.Turn),
                board.fullmove_number,
                bool(board.has_kingside_castling_rights(True)),   # True = White
                bool(board.has_queenside_castling_rights(True)),
                bool(board.has_kingside_castling_rights(False)),  # False = Black
                bool(board.has_queenside_castling_rights(False)),
                board.halfmove_clock
            ]
        )